#### Подключаем библиотеки

In [1]:
# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

# Important for pipeline normal working
import sklearn
sklearn.set_config(transform_output="pandas")

# Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, PowerTransformer

#### Настройка отображения для pandas

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

#### Загрузка данных

In [3]:
train = pd.read_csv('../data_from_kaggle/train.csv', index_col="Id")
test = pd.read_csv('../data_from_kaggle/test.csv', index_col="Id")

#### Разделение на target и train

In [4]:
X_train, y = train.drop('SalePrice', axis=1), train['SalePrice']

#### Объединение test и train выборок, чтобы они прошли препроцессинг

In [5]:
all_data = pd.concat([X_train, test])
all_data.shape

(2919, 79)

#### Просмотр пропущенных данных

#### Добавляем фичи тут, описаны они будут позже
Average_Qual - Общее состояние дома, состоящее из общего качества отделки и общего состояния дома

CompAge - "Возраст" дома, сотоящий из разницы между годом строительства и постройки (что очевидно)

ReNew - "Возраст" реновации - Разница между годом строительства и ремонта

Baths - общее количество ванных на дом

Area - Общая облагороженная площадь дома

HQArea - "Качественная" площадь для жилья.

In [6]:
def add_col(X):
    X['Average_Qual'] = X['OverallQual'] + X['OverallCond']
    X['CompAge'] = X['YrSold'].astype(int) - X['YearBuilt'].astype(int)
    X['ReNew'] = X['YearRemodAdd'].astype(int) - X['YearBuilt'].astype(int)
    X['Baths'] = X['BsmtFullBath'] + X['FullBath'] + (0.5 * X['HalfBath']) + (0.5 * X['BsmtHalfBath'])
    X['Area'] = X['GrLivArea'] + X['1stFlrSF'] + X['2ndFlrSF'] + 0.5 * X['GarageArea'] + 0.5 * X['TotalBsmtSF'] + 1 * X['MasVnrArea']
    X['HQArea'] = X['1stFlrSF'] + X['2ndFlrSF'] + X['TotalBsmtSF']
    return X

add_col(all_data)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Average_Qual,CompAge,ReNew,Baths,Area,HQArea
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,12,5,0,3.5,4318.0,2566.0
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,14,31,0,2.5,3385.0,2524.0
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,12,7,1,3.5,4498.0,2706.0
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,12,91,55,2.0,4133.0,2473.0
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,13,8,0,3.5,5736.5,3343.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,11,36,0,1.5,2457.0,1638.0
2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,9,36,0,1.5,2600.0,1638.0
2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf

In [7]:
missing_val_count_by_column = (all_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType      1766
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
Baths              2
Area              25
HQArea             1
dtype: int64


In [ ]:
NA_cols = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

all_data[NA_cols].isna().sum()

Alley           2721
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
FireplaceQu     1420
GarageType       157
GarageFinish     159
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
dtype: int64

#### Перевод в другой тип данных

In [9]:
all_data['MSSubClass'] = all_data['MSSubClass'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
all_data['OverallQual'] = all_data['OverallQual'].astype(str)

#### Разбитие на желаемое заполнение

In [10]:
num_cols_zero = ['GarageYrBlt', 'GarageCars', 'GarageArea']
num_cols_most_freq = ['MasVnrArea', 'HQArea','BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']
num_cols_mean = ['LotFrontage', 'Baths']
num_cols_meadian = ['Area']

#### Разделение на категориальные и числовые колонки для удобства

In [11]:
categorical_cols = [col_name for col_name in all_data.columns if all_data[col_name].dtype == 'object']
numerical_cols = [col_name for col_name in all_data.columns if all_data[col_name].dtype != 'object']

#### Просмотр количества пропущенных и названия столбцов для каждой категории

In [12]:
# Для числовых столбцов

for cols in numerical_cols:
    miss = all_data[cols].isna().sum()
    if miss > 0:
        print(f'{cols}: {miss}')

LotFrontage: 486
MasVnrArea: 23
BsmtFinSF1: 1
BsmtFinSF2: 1
BsmtUnfSF: 1
TotalBsmtSF: 1
BsmtFullBath: 2
BsmtHalfBath: 2
GarageYrBlt: 159
GarageCars: 1
GarageArea: 1
Baths: 2
Area: 25
HQArea: 1


In [ ]:
# Для категориальных столбцов

for cols in categorical_cols:
    missing = all_data[cols].isna().sum()
    if missing > 0:
        print(f'{cols}: {missing}')

MSZoning: 4
Alley: 2721
Utilities: 2
Exterior1st: 1
Exterior2nd: 1
MasVnrType: 1766
BsmtQual: 81
BsmtCond: 82
BsmtExposure: 82
BsmtFinType1: 79
BsmtFinType2: 80
Electrical: 1
KitchenQual: 1
Functional: 2
FireplaceQu: 1420
GarageType: 157
GarageFinish: 159
GarageQual: 159
GarageCond: 159
PoolQC: 2909
Fence: 2348
MiscFeature: 2814
SaleType: 1


#### Выделение категориальных столбцов с NaN

In [14]:
NaN_cols = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Electrical', 'KitchenQual', 'Functional', 'SaleType']

#### Разделение на NaN и NA

In [15]:
cat_cols_NA = NA_cols
cat_cols_NaN = NaN_cols

#### Построение Imputer'a с помощью ColumnTransformer

In [17]:
# Заполняем категориальные данные
categorical_imputer_NaN = SimpleImputer(strategy='most_frequent')
categorical_imputer_NA = SimpleImputer(strategy="constant", fill_value='NA')

# Заполняем цифровые данные
numerical_imputer_mean = SimpleImputer(strategy="mean")
numerical_imputer_mf = SimpleImputer(strategy="most_frequent")
numerical_imputer_0 = SimpleImputer(strategy='constant', fill_value=0)
numerical_imputer_median = SimpleImputer(strategy='median')

imputer = ColumnTransformer(
    transformers = [
        ('num_imp_mean', numerical_imputer_mean, num_cols_mean),
        ('num_imp_mf', numerical_imputer_mf, num_cols_most_freq),
        ('num_imp_zero', numerical_imputer_0, num_cols_zero),
        ('num_imp_median', numerical_imputer_median, num_cols_meadian),
        ('cat_imp_NA', categorical_imputer_NA, cat_cols_NA),
        ('cat_imp_NaN', categorical_imputer_NaN, cat_cols_NaN)
    ],
    verbose_feature_names_out = False,
    remainder = "passthrough"
)

#### Делаем список колонок для Encoder|Scaler


In [18]:
# columns for encoding
col_for_enc = [col_name for col_name in all_data.columns if all_data[col_name].dtype == "object"]
# columns for scaling
col_for_scale = [col_name for col_name in all_data.columns if all_data[col_name].dtype != "object"]

In [19]:
col_for_enc

['MSSubClass',
 'MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition']

[
 'MSZoning',



 'LandContour',
 'Utilities',
 'LotConfig',

 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',

 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',

 'Foundation',



 'Heating',

 'Electrical',


 'GarageType',

 




 'MiscFeature',


 'SaleType',
 'SaleCondition']

#### Формируем Scaler/Encoder

In [20]:
# encoders
one_hot_encoder = OneHotEncoder(sparse_output=False)
#cat_boost_enc = CatBoostEncoder()

#scaler
PT = PowerTransformer(method='yeo-johnson')

# pipeline for encoder and scaler
encoder_scaler = ColumnTransformer(
    [
        ("one_hot_encoding", one_hot_encoder, col_for_enc),
        ('Normalize', PT, col_for_scale)
    ],
    verbose_feature_names_out = False,
    remainder = 'passthrough'
)

#### Делаем общий Pipeline

In [21]:
preprocessor = Pipeline(
    [
        ("imputer", imputer),
        ("encoder", encoder_scaler)    
    ]
)

In [22]:
preprocessor

Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_imp_mean',
                                                  SimpleImputer(),
                                                  ['LotFrontage', 'Baths']),
                                                 ('num_imp_mf',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['MasVnrArea', 'HQArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath']),
                                                 ('num_imp_zero',
                                                  SimpleImputer(fill_value=0,
                                                                strate...
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                                                   'GrLivArea', 'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBath', 'BedroomAbvGr',
                                                   'KitchenAbvGr',
                                                   'TotRmsAbvGrd', 'Fireplaces',
                                                   'GarageYrBlt', 'GarageCars',
                                                   'GarageArea', 'WoodDeckSF',
                                                   'OpenPorchSF',
                                                   'EnclosedPorch', '3SsnPorch',
                                                   'ScreenPorch', 'PoolArea', ...])],
                                   verbose_feature_names_out=False))])

#### Прогоняем наши данные через Pipeline обработки данных

In [23]:
data_piped = preprocessor.fit_transform(all_data)

/home/marena/miniforge3/envs/myenv/lib/python3.12/site-packages/numpy/core/_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


#### Объединяем данные в 2 датасета, проверяем на наличие пропусков и сохраняем, для дальнейшего использования в файле tmodels.ipynb
target - это прологарифмированная колонка y из датасета train 

data - это датасет all_data 

In [24]:
target = y
target_log = np.log(target)

data_piped.to_csv('../data_piped/data.csv')

target_log.to_csv('../data_piped/target.csv')